<a href="https://colab.research.google.com/github/brem-21/RAG-APP/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#%%
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
from uuid import uuid4
import requests
import asyncio
import chromadb
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
import torch

/home/kali/GENAI/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def max_token_length(txt_list:list):
    max_length = 0
    for txt in txt_list:
        token_count = len(re.findall(r'\w+', txt))
        if token_count > max_length:
            max_length = token_count
    return f"Max Token Length: {max_length} tokens"

In [3]:
model_max_chunk_length = 256
token_splitter = SentenceTransformersTokenTextSplitter(
    tokens_per_chunk=model_max_chunk_length,
    model_name="all-MiniLM-L6-v2",
    chunk_overlap=0
)

In [4]:
text_path = "paulgraham.txt"
with open(text_path, "r", encoding="utf-8") as f:
    text_raw=f.read()

In [6]:
character_splitter = RecursiveCharacterTextSplitter(
    separators=['\n  \n', '\n\n', '\n', '. '],
    chunk_size=1000,
    chunk_overlap=0
)
text_splitted = character_splitter.split_text(text_raw)

In [7]:
max_token_length(text_splitted)

'Max Token Length: 205 tokens'

In [8]:
print(f"Total number of splitted chunks: {len(text_splitted)}")

Total number of splitted chunks: 103


In [9]:
text_tokens = []
for text in text_splitted:
    text_tokens.extend(token_splitter.split_text(text))
print(f"Total number of tokens: {len(text_tokens)}")

Total number of tokens: 103


In [10]:
max_token_length(text_tokens)

'Max Token Length: 205 tokens'

In [11]:
embedding_fn = SentenceTransformerEmbeddingFunction(
    model_name="all-MiniLM-L6-v2",
    device="cpu"
)

In [12]:
len(embedding_fn(text_tokens[0])[0])

384

In [13]:
chroma_db = chromadb.Client()
chroma_collection = chroma_db.get_or_create_collection("paul_graham", embedding_function=embedding_fn)

In [14]:
ids = [str(uuid4()) for _ in range(len(text_tokens))]
ids[:5]

['e83b6a83-2a87-42d1-8a05-6dd4498f326c',
 '130a58b4-2bfe-4cd5-9bd0-9a4e36f30007',
 '5459f3a3-c32e-4f4b-ab94-bf41a17de90e',
 '0b4009f2-9c0b-4788-aff8-691ac52349b8',
 'a17adcb2-3ade-41b6-8988-cd90cafa9c35']

In [15]:
chroma_collection.add(documents=text_tokens, ids=ids)

In [16]:
res = chroma_collection.query(query_texts=["What did paul graham work on"], n_results = 10)
res['documents']

[['in september, robert rebelled. " we \' ve been working on this for a month, " he said, " and it \' s still not done. " this is funny in retrospect, because he would still be working on it almost 3 years later. but i decided it might be prudent to recruit more programmers, and i asked robert who else in grad school with him was really good. he recommended trevor blackwell, which surprised me at first, because at that point i knew trevor mainly for his plan to reduce everything in his life to a stack of notecards, which he carried around with him. but rtm was right, as usual. trevor turned out to be a frighteningly effective hacker. it was a lot of fun working with robert and trevor. they \' re the two most independent - minded people i know, and in completely different ways. if you could see inside rtm \' s brain it would look like a colonial new england church, and if you could see inside trevor \' s it would look like the worst excesses of austrian rococo.',
  "as jessica and i wer

In [17]:
chroma_collection = chroma_db.get_or_create_collection("paul_graham", embedding_function=embedding_fn)

In [18]:
def get_query_results(query_text: str, n_results: int = 5) -> str:
    """Retrieve relevant information from ChromaDB."""
    res = chroma_collection.query(query_texts=[query_text], n_results=n_results)
    docs = res["documents"][0]

    # Check if 'metadatas' exists, is a list, and contains valid metadata dictionaries
    if res.get('metadatas') and isinstance(res['metadatas'], list) and len(res['metadatas']) > 0:
        keywords = [item.get('keyword', '') if isinstance(item, dict) else '' for item in res['metadatas'][0]]
    else:
        keywords = [''] * len(docs)  # Assign empty keywords if metadata is missing

    return '; '.join([f'{keyword}: {information}' for keyword, information in zip(keywords, docs)])


In [37]:
import google.generativeai as genai
import os
from dotenv import load_dotenv
load_dotenv()

# Set up Google API key
google_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=google_api_key)

def rag_tool(user_query: str) -> str:
    """Retrieves relevant data from the database and generates a response using Gemini-2.0-Flash."""
    retrieved_results = get_query_results(user_query)
    
    system_prompt = (
        "You are an AI assistant with RAG capabilities. You will be given a user query and relevant retrieved documents. "
        "Please generate a response based only on the provided information."
    )

    full_query = f"User Query: {user_query}\n\nRetrieved Documents:\n{retrieved_results}\n\nInstruction: {system_prompt}"

    # Initialize the Gemini model
    model = genai.GenerativeModel("gemini-2.0-flash")

    # Generate response
    response = model.generate_content(full_query)

    return response.text


In [40]:
async def main():
    query = "after college.??"
    response = rag_tool(query)  # Directly call the RAG tool
    print("Response from RAG:")
    print("----------------------------")
    print(response)

if __name__ == "__main__":
    # Remove asyncio.run() and directly await the main function
    await main()

Response from RAG:
----------------------------
After college, the author:

*   Visited Rich Draves at CMU and realized the potential of making art that lasts, like paintings.
*   Applied to art schools, including RISD and the Accademia di Belli Arti in Florence, and attended RISD as a transfer sophomore, requiring them to do the foundation program.
*   Started taking art classes at Harvard while in a PhD program in computer science.
*   Resumed their old life in New York but with the added benefit of being rich, which allowed them to take taxis and eat at restaurants.

